In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
import torch
from torch import nn
from dsin.ae.data_manager.data_loader import (
    SideinformationImageImageList, ImageSiTuple)
from dsin.ae import config
from dsin.ae.si_ae import SideInformationAutoEncoder
from dsin.ae.si_net import SiNetChannelIn
from dsin.ae.loss_man import LossManager
from dsin.ae.distortions import Distortions, DistTypes
from dsin.ae.kitti_normalizer import ChangeImageStatsToKitti, ChangeState
from dsin.ae import config


In [3]:
config.H_target = 2* 0.3
config.autoencoder_loss_distortion_to_minimize=DistTypes.MS_SSMIM
config.beta

500

In [4]:
class AverageMetric(Callback):
    "Wrap a `func` in a callback for metrics computation."
    def __init__(self, func,name):
        # If it's a partial, use func.func
        #         name = getattr(func,'func',func).__name__
        self.func, self.name = func, name

    def on_epoch_begin(self, **kwargs):
        "Set the inner value to 0."
        self.val, self.count = 0.,0

    def on_batch_end(self, last_output, last_target, **kwargs):
        "Update metric computation with `last_output` and `last_target`."
        if not is_listy(last_target): last_target=[last_target]
        self.count += last_target[0].size(0) # batch size
        
        if last_output[0] is None:
            X_OUTPUT = 1 # X_DEC
        else:
            X_OUTPUT = 0 # X_HAT_OUT
        val = self.func(last_output[X_OUTPUT], last_target[0])
        self.val += last_target[0].size(0) * val.detach().cpu()
        
    
    
    def on_epoch_end(self, last_metrics, **kwargs):
        "Set the final result in `last_metrics`."
        return add_metrics(last_metrics, self.val/self.count)
    
class ParameterMetricCallback(Callback):
    def __init__(self,loss_man):
        self.loss_man = loss_man
    
    def on_backward_begin(self,*args, **kwargs):
       
        if hasattr(self.loss_man,'soft_bit_entropy') and :
            self.pbar.child.comment += f' soft_bit_entropy: {self.loss_man.soft_bit_entropy}'


In [5]:
config.use_si_flag = SiNetChannelIn.WithSideInformation
config.use_si_flag

<SiNetChannelIn.WithSideInformation: 6>

In [6]:
torch.__version__

'1.5.1'

In [7]:
si_autoencoder = SideInformationAutoEncoder(config.use_si_flag)
path = "../src/dsin/data"
pct= 1 #0.0005 #0.25

valid_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_baseline_val.txt"],pct=pct)
train_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_baseline_train.txt"],pct=pct)

image_lists = ItemLists(
    path=path, train=train_image_list, valid=valid_image_list)

# ll = image_lists.label_from_func(lambda x: x)

tfms = None #get_transforms(do_flip=True, max_rotate=0.0)
batchsize = 1
data = (image_lists
        .label_from_func(lambda x: x)
        .transform(tfms, size=(336, 1224), resize_method=ResizeMethod.CROP, tfm_y=True)
        .databunch(bs=batchsize))
learn = Learner(data=data,
                     model=si_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(config.use_si_flag),
                     metrics=[AverageMetric(Distortions._calc_dist,"MS_SSIM")])



In [ ]:
learn.model.true_tuple_loss_false_just_out = True
learn.model.use_side_infomation = SiNetChannelIn.NoSideInformation
learn.loss_func.use_side_infomation = SiNetChannelIn.NoSideInformation
learn.fit(1, lr=0.0005,wd=0.00001,callbacks=[ParameterMetricCallback()])


epoch,train_loss,valid_loss,MS_SSIM,time


> <ipython-input-4-8e39cd9df5ff>(34)on_backward_begin()
-> val = kwargs["metrics"][0].val
(Pdb) kwargs
{'epoch': 0, 'iteration': 0, 'num_batch': 0, 'skip_validate': False, 'n_epochs': 1, 'pbar': <fastprogress.fastprogress.NBMasterBar object at 0x7f297e5ed4d0>, 'metrics': [AverageMetric
func: <function Distortions._calc_dist at 0x7f297e61ec20>
name: MS_SSIM], 'stop_training': False, 'last_input': [tensor([[[[0.4142, 0.3800, 0.3353,  ..., 0.4504, 0.4510, 0.4386],
          [0.4181, 0.3967, 0.3509,  ..., 0.4147, 0.4128, 0.3843],
          [0.3496, 0.3492, 0.3253,  ..., 0.3733, 0.3673, 0.3328],
          ...,
          [0.6858, 0.6709, 0.6475,  ..., 0.0785, 0.0774, 0.0793],
          [0.6723, 0.6464, 0.6199,  ..., 0.0724, 0.0725, 0.0735],
          [0.6145, 0.5988, 0.5775,  ..., 0.0675, 0.0676, 0.0676]],

         [[0.3810, 0.4412, 0.4110,  ..., 0.4690, 0.5274, 0.5623],
          [0.3071, 0.3698, 0.3556,  ..., 0.3649, 0.4461, 0.4780],
          [0.2846, 0.3316, 0.3326,  ..., 0.4279, 0.5060

In [ ]:
learn.data.train_ds[2][0].img

In [ ]:
learn.data.train_ds[1][0].si_img

In [ ]:
learn.loss_func.soft_bit_entropy

In [ ]:
learn.loss_func.autoencoder_loss_value

In [ ]:

# # show_image(Image(learn.model.my_tuple[0][:,:50,:50]/255))
# # Image(learn.model.my_tuple[2].squeeze_().detach()/255.0)
mt = learn.model.my_tuple[3].squeeze_().detach()
mx, mn =torch.max(mt), torch.min(mt)
diff = mx-mn
show_image(Image((mt - mn)/(diff)),figsize=(30,30))
# # print(mn)

# torch.min(learn.model.my_tuple[2].squeeze_().detach())

In [ ]:
# learn.model.true_tuple_loss_false_just_out = False

# learn.show_results(figsize=(30,30))